In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [3]:
# Ratings data
ratings = tfds.load('movielens/100k-ratings', split='train')
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split = 'train')

In [4]:
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7], dtype=int64),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [5]:
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_genres': array([4], dtype=int64),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [6]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [7]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [8]:
# Figure out unique user ids and movie titles present in the data
# Map the raw values of our categorical features to embedding vectors in our model
# Need a vocabulary that maps a raw feature value to an integer in a continguous range:
# Allow us to look up the corresponding embeddings in our embedding tables

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x['user_id'])
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [9]:
unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

# Implementing a model

- Choosing the architecture of our model is a key part of modelling
- Build each tower separately and then combine them in the final model

## The query tower


- Decide on the dimensionality of the query and candidate representations
    - Higher values - More accurate, slower to fit and more prone to overfitting

In [10]:
embedding_dimension = 32

- Define the model itself
    - Use Keras preprocessing layers
        - Convert user ids to integers
        - Convert those to user embeddings via an Embedding layer

In [11]:
user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary = unique_user_ids, mask_token = None),
    # We add an additonal embedding to account for unknown tokens
    tf.keras.layers.Embedding(
        len(unique_user_ids) + 1, embedding_dimension
    )
    ])

## The candidate tower

- Same process

In [12]:
movie_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary = unique_movie_titles, mask_token = None
    ),
    tf.keras.layers.Embedding(
        len(unique_movie_titles) + 1, embedding_dimension
    )
])

## Metrics

- Positive (user, movie) pairs

- Compare the affinity score:
    - The model calculates for this pair
    - The scores of all the other possible candidates
        - Score for the positive pair is higher than for all other candidates
        - Model is highly accurate

- Computes metrics for across top K candidates surfaced by a retrieval model

In [13]:
metrics = tfrs.metrics.FactorizedTopK(
    candidates = movies.batch(128).map(movie_model)
)

## Loss

- TFRS has several loss layers and tasks to make this easy
- Use Retrieval task object:
    - A convenience wrapper
    - Bundles together the loss function and metric computation

- Task itself is a Keras layer
    - Takes the query and candidate embeddings as arguments
    - Returns the computed loss

In [14]:
task = tfrs.tasks.Retrieval(
    metrics = metrics
) 

## The full model

In [19]:
class MovielensModel(tfrs.Model):
    def __init__(self, user_model, movie_model):
        super().__init__()
        self.movie_model: tf.keras.Model = movie_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task
    
    def compute_loss(self, features: Dict[Text, tf.Tensor], training = False) -> tf.Tensor:
        # Pick out user features and pass them into the user model
        user_embeddings = self.user_model(features['user_id'])
        # Pick out the movie features and pass them into the movie model,
        # getting embeddings back
        positive_movie_embeddings = self.movie_model(features['movie_title'])
        # Task computes the loss and the metrics
        return self.task(user_embeddings, positive_movie_embeddings)

- tfrs.Model base class
    - Convenience class
    - Compute both training and test losses using the same method
(Still a plain Keras model, achieve the same functionality by inheriting from tf.keras.Model
and overriding the train_step and test_step functions)

## Fitting and evaluating

In [20]:
# Instantitate the model
model = MovielensModel(user_model, movie_model)
model.compile(optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [21]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [29]:
model.fit(cached_train, epochs = 3)

Epoch 1/3


10/10 [==============================] - 5s 506ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0015 - factorized_top_k/top_5_categorical_accuracy: 0.0364 - factorized_top_k/top_10_categorical_accuracy: 0.0730 - factorized_top_k/top_50_categorical_accuracy: 0.2704 - factorized_top_k/top_100_categorical_accuracy: 0.4216 - loss: 62688.2493 - regularization_loss: 0.0000e+00 - total_loss: 62688.2493
Epoch 2/3
10/10 [==============================] - 5s 502ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0014 - factorized_top_k/top_5_categorical_accuracy: 0.0362 - factorized_top_k/top_10_categorical_accuracy: 0.0729 - factorized_top_k/top_50_categorical_accuracy: 0.2715 - factorized_top_k/top_100_categorical_accuracy: 0.4233 - loss: 62652.0312 - regularization_loss: 0.0000e+00 - total_loss: 62652.0312
Epoch 3/3
10/10 [==============================] - 5s 499ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0015 - factorized_top_k/top_5_categorical_accuracy: 0.0368 - f

In [26]:
model.evaluate(cached_test, return_dict = True)

5/5 [==============================] - 1s 161ms/step - factorized_top_k/top_1_categorical_accuracy: 1.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0017 - factorized_top_k/top_10_categorical_accuracy: 0.0050 - factorized_top_k/top_50_categorical_accuracy: 0.0697 - factorized_top_k/top_100_categorical_accuracy: 0.1664 - loss: 32348.0573 - regularization_loss: 0.0000e+00 - total_loss: 32348.0573


{'factorized_top_k/top_1_categorical_accuracy': 0.0001500000071246177,
 'factorized_top_k/top_5_categorical_accuracy': 0.0017000000225380063,
 'factorized_top_k/top_10_categorical_accuracy': 0.005049999803304672,
 'factorized_top_k/top_50_categorical_accuracy': 0.06965000182390213,
 'factorized_top_k/top_100_categorical_accuracy': 0.1664000004529953,
 'loss': 29337.298828125,
 'regularization_loss': 0,
 'total_loss': 29337.298828125}

- Performance is worse than training performance
    - Perform better on the data that is has seen 
    (Overfitting is strong when models have many parameters)
    (Can be mediated by model regularization and use of user and movie features that help the model generalize better to unseen data)
    - The model is re-recommending some of users' already watched movies
    (Known-positive watches can crowd out test movies out of top K recommendations)
    (Tackled by excluding previously seen movies from test recommendations)

## Making predictions

In [28]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
    tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)
# Get recommendations
_, titles = index(tf.constant(['42']))
print(f'Recommendations for user 42: {titles[0, :3]}')

Recommendations for user 42: [b'Rent-a-Kid (1995)' b'Paper, The (1994)' b'Bronx Tale, A (1993)']
